# import

In [ ]:
import pandas as pd
import numpy as np
import json
import datetime

# date

In [ ]:
dt=datetime.datetime.now()

# inital data

In [ ]:
df=pd.read_csv("data.csv")

In [ ]:
df_f=df[["id","contracts"]]
df_f["contracts"]=~df_f["contracts"].isna()

In [ ]:
df_j=df["contracts"]
df_j=df_j.fillna("[]")
df_j=df_j.where(df_j.isna(),df_j.apply(lambda x: json.loads(x)),axis=0)

In [ ]:
df["contracts"]=df_j

In [ ]:
df=df.explode("contracts")
df=df.reset_index()
df=df[["id","date_start"]].join(pd.json_normalize(df["contracts"]))

In [ ]:
df.loc[df["loan_summa"]=="",["loan_summa"]]=0

In [ ]:
df["date_start"]=df["date_start"].str[:10]

In [ ]:
df["date_start"]=pd.to_datetime(df["date_start"],format="%Y-%m-%d")
df["claim_date"]=pd.to_datetime(df["claim_date"],format="%d.%m.%Y")
df["contract_date"]=pd.to_datetime(df["contract_date"],format="%d.%m.%Y")
df["loan_summa"]=df["loan_summa"].astype("double")


In [ ]:
df.loc[df["claim_date"]>df["date_start"],["claim_date"]]=df["date_start"]
df.loc[df["contract_date"]>df["date_start"],["contract_date"]]=df["date_start"]

In [ ]:
# df.loc[df["contract_date"].dt.strftime("%a")=='Sat',"contract_date"]=df["contract_date"]+datetime.timedelta(days=2)
# df.loc[df["contract_date"].dt.strftime("%a")=='Sun',"contract_date"]=df["contract_date"]+datetime.timedelta(days=1)

# df.loc[df["claim_date"].dt.strftime("%a")=='Sat',"claim_date"]=df["claim_date"]+datetime.timedelta(days=2)
# df.loc[df["claim_date"].dt.strftime("%a")=='Sun',"claim_date"]=df["claim_date"]+datetime.timedelta(days=1)

# tot_claim_cnt_l180d

In [ ]:
df1=df.loc[:,["id","claim_id","claim_date"]]
df1=df1[df1["claim_date"]>dt-datetime.timedelta(days=180)]
df1=df1.loc[~df1.claim_date.isna(),["id","claim_id"]]

In [ ]:
df1=df1.groupby("id").count().reset_index()

In [ ]:
df1=df1.rename(columns={"claim_id":"tot_claim_cnt_l180d"})

# disb_active_bank_loan_wo_tbc

In [ ]:
df_2=df[~df["contract_date"].isnull() & ~df["bank"].isin(['LIZ', 'LOM', 'MKO', 'SUG', None])]

In [ ]:
df2=df_2.groupby("id").agg({"loan_summa":["sum"]}).loan_summa.reset_index()

In [ ]:
df2=df2.rename(columns={"sum":"disb_active_bank_loan_wo_tbc"})

# day_sinlastloan

In [ ]:
df3=df.loc[~df.summa.isna(),["id","summa","contract_date"]]
df3=df3[df3["summa"]!=""]

In [ ]:
df3["days_diff"]=dt-df3["contract_date"]

In [ ]:
df3["days_diff"]=df3["days_diff"]/ np.timedelta64(1, 'D')

In [ ]:
df3=df3.groupby("id")["days_diff"].min().reset_index()

In [ ]:
df3=df3.rename(columns={"days_diff":"day_sinlastloan"})

# join

In [ ]:
df_f=df_f.merge(df1,left_on="id",right_on="id",how="left")
df_f=df_f.merge(df2,left_on="id",right_on="id",how="left")
df_f=df_f.merge(df3,left_on="id",right_on="id",how="left")

In [ ]:
df_f[["disb_active_bank_loan_wo_tbc","day_sinlastloan"]]=df_f[["disb_active_bank_loan_wo_tbc","day_sinlastloan"]].fillna(-1)

In [ ]:
df_f.loc[~df_f["contracts"],"disb_active_bank_loan_wo_tbc"]=-3
df_f.loc[~df_f["contracts"],"day_sinlastloan"]=-3
df_f.loc[~df_f["contracts"],"tot_claim_cnt_l180d"]=-3

In [ ]:
df_f=df_f.drop("contracts",axis=1)

# write

In [ ]:
df_f.to_csv("contract_features.csv",index=False)